In [1]:
cd ..

/master


In [2]:
%autosave 600
%matplotlib inline
#%load_ext autoreload
#%autoreload 2
#%pdb

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""

import sys
sys.path.append("notebooks")

import numpy as np
import matplotlib.pyplot as plt
import torch
torch.set_num_threads(4)

Autosaving every 600 seconds


# Neural Process Images

last update: 08 June 2019

**Aim**: This notebook shows how to train and sample from a Neural Process for a simple class of 1D functions. The functions are defined by $f(x) = a * sin(x - b)$ where $a$ and $b$ and randomly sampled.

**Credits**: the task, visualisation and notebook structure are taken from : https://github.com/EmilienDupont/neural-processes
